In [1]:
import numpy as np
import json
import os
import cv2

# 🔥 IMPORTAR LA VERSIÓN CORREGIDA
from neural_network import RedNeuronal

print("🔍 Métodos disponibles en RedNeuronal:")
print(dir(RedNeuronal))


class ImageProcessor:
    """Procesador de imágenes simplificado"""
    
    def __init__(self, target_size=(600, 800)):
        self.target_size = target_size
    
    def load_and_preprocess(self, image_path: str) -> np.ndarray:
        """Cargar y preprocesar una imagen"""
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"No se encontró la imagen: {image_path}")
        
        # Leer imagen
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        
        # Redimensionar
        img = cv2.resize(img, self.target_size)
        
        # Convertir a escala de grises
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Normalizar píxeles (0-1)
        img_normalized = img_gray.astype(np.float32) / 255.0
        
        # Aplanar la imagen
        img_flattened = img_normalized.flatten()
        
        return img_flattened

def load_model_info(info_path: str) -> dict:
    """Cargar información del modelo"""
    with open(info_path, 'r') as f:
        return json.load(f)

def test_single_image(model_path: str, info_path: str, image_path: str):
    """Probar el modelo con una sola imagen"""
    print(f"🔍 Probando imagen: {image_path}")
    
    try:
        # Cargar información del modelo
        model_info = load_model_info(info_path)
        class_names = model_info['class_names']
        target_size = tuple(model_info['target_size'])
        
        print(f"📊 Clases del modelo: {class_names}")
        print(f"📐 Tamaño objetivo: {target_size}")
        
        # 🔥 CREAR Y CARGAR MODELO (AHORA CON load_model())
        red = RedNeuronal(model_info['architecture'])
        red.load_model(model_path)
        
        # Procesar imagen
        processor = ImageProcessor(target_size=target_size)
        img_processed = processor.load_and_preprocess(image_path)
        
        print(f"🖼️  Imagen procesada: {img_processed.shape} píxeles")
        
        # ✅ HACER PREDICCIÓN CORRECTAMENTE
        img_batch = np.array([img_processed])  # Añadir dimensión batch
        prediction = red.predict(img_batch)[0]  # Obtener primera predicción
        
        # Mostrar resultados
        predicted_class_idx = np.argmax(prediction)
        predicted_class = class_names[predicted_class_idx]
        confidence = prediction[predicted_class_idx]
        
        print(f"\n📊 Resultados de la predicción:")
        print(f"   - Clase predicha: {predicted_class}")
        print(f"   - Confianza: {confidence:.4f} ({confidence*100:.2f}%)")
        print()
        
        print("📈 Probabilidades por clase:")
        for i, class_name in enumerate(class_names):
            prob = prediction[i]
            bar = "█" * int(prob * 20)
            print(f"   {class_name:12}: {prob:.4f} |{bar:<20}|")
        
        return predicted_class, confidence
        
    except Exception as e:
        print(f"❌ Error procesando la imagen: {e}")
        import traceback
        traceback.print_exc()
        return None, 0

def main():
    """Función principal simplificada"""
    print("=== Prueba de Red Neuronal - VERSIÓN CORREGIDA ===")
    print()
    
    # Buscar archivos del modelo
    model_files = [f for f in os.listdir('.') if f.endswith('.json') and 'modelo' in f.lower()]
    info_files = [f for f in os.listdir('.') if 'info' in f.lower() and f.endswith('.json')]
    
    print(f"📁 Archivos encontrados:")
    print(f"   - Modelos: {model_files}")
    print(f"   - Info: {info_files}")
    
    if not model_files:
        print("❌ No se encontraron modelos entrenados")
        return
    
    if not info_files:
        print("❌ No se encontraron archivos de información")
        return
    
    # Usar el primer modelo encontrado
    model_path = model_files[1]
    info_path = info_files[0]
    
    print(f"\n✅ Usando:")
    print(f"   - Modelo: {model_path}")
    print(f"   - Info: {info_path}")
    print()
    
    # Probar con una imagen
    while True:
        image_path = input("🖼️  Ruta de la imagen (o 'salir'): ").strip()
        
        if image_path.lower() == 'salir':
            print("👋 ¡Hasta luego!")
            break
        
        if os.path.exists(image_path):
            test_single_image(model_path, info_path, image_path)
        else:
            print("❌ No se encontró la imagen")
        
        print("\n" + "="*50 + "\n")

if __name__ == "__main__":
    main()


🔍 Métodos disponibles en RedNeuronal:
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_activation', '_activation_derivative', '_backward', '_cross_entropy', '_forward', 'evaluate', 'load_model', 'predict', 'save_model', 'train']
=== Prueba de Red Neuronal - VERSIÓN CORREGIDA ===

📁 Archivos encontrados:
   - Modelos: ['modelo_info.json', 'modelo_reconocimiento_imagenes.json']
   - Info: ['modelo_info.json']

✅ Usando:
   - Modelo: modelo_reconocimiento_imagenes.json
   - Info: modelo_info.json



🖼️  Ruta de la imagen (o 'salir'):  C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\imagen_prueba.jpg


🔍 Probando imagen: C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\imagen_prueba.jpg
📊 Clases del modelo: ['Darwin', 'No_Darwin']
📐 Tamaño objetivo: (600, 800)
✅ Modelo cargado desde: modelo_reconocimiento_imagenes.json
📊 Arquitectura: 4 capas
🖼️  Imagen procesada: (480000,) píxeles

📊 Resultados de la predicción:
   - Clase predicha: No_Darwin
   - Confianza: 0.9999 (99.99%)

📈 Probabilidades por clase:
   Darwin      : 0.0001 |                    |
   No_Darwin   : 0.9999 |███████████████████ |




🖼️  Ruta de la imagen (o 'salir'):  C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\ima_p2.jpg.jpg


❌ No se encontró la imagen




...

❌ No se encontró la imagen




🖼️  Ruta de la imagen (o 'salir'):  C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\ima_p3.jpg


🔍 Probando imagen: C:\Users\darwi\proyectosAnaconda\ProyectoNeuronal\ima_p3.jpg
📊 Clases del modelo: ['Darwin', 'No_Darwin']
📐 Tamaño objetivo: (600, 800)
✅ Modelo cargado desde: modelo_reconocimiento_imagenes.json
📊 Arquitectura: 4 capas
🖼️  Imagen procesada: (480000,) píxeles

📊 Resultados de la predicción:
   - Clase predicha: Darwin
   - Confianza: 0.9982 (99.82%)

📈 Probabilidades por clase:
   Darwin      : 0.9982 |███████████████████ |
   No_Darwin   : 0.0018 |                    |




🖼️  Ruta de la imagen (o 'salir'):  salir


👋 ¡Hasta luego!
